In [8]:
%tensorflow_version 1.x

In [2]:
!pip list | grep tensorflow

mesh-tensorflow               0.1.12
tensorflow                    1.15.2
tensorflow-datasets           4.0.1
tensorflow-estimator          1.15.1
tensorflow-gan                2.0.0
tensorflow-gcs-config         2.8.0
tensorflow-hub                0.12.0
tensorflow-io-gcs-filesystem  0.24.0
tensorflow-metadata           1.6.0
tensorflow-probability        0.7.0


In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
!mkdir "/content/gdrive/MyDrive/Colab Notebooks/bert-pretrain-v2"
%cd "/content/gdrive/MyDrive/Colab Notebooks/bert-pretrain-v2"

/content/gdrive/MyDrive/Colab Notebooks/bert-pretrain-v2


KcBERT Dataset
- "공개된 한국어 BERT는 대부분 한국어 위키, 뉴스 기사, 책 등 잘 정제된 데이터를 기반으로 학습한 모델입니다. 한편, 실제로 NSMC와 같은 댓글형 데이터셋은 정제되지 않았고 구어체 특징에 신조어가 많으며, 오탈자 등 공식적인 글쓰기에서 나타나지 않는 표현들이 빈번하게 등장합니다.KcBERT는 위와 같은 특성의 데이터셋에 적용하기 위해, 네이버 뉴스에서 댓글과 대댓글을 수집해, 토크나이저와 BERT모델을 처음부터 학습한 Pretrained BERT 모델

In [5]:
!pip install -q Korpora
!pip install -q tokenizers
!git clone -q https://github.com/google-research/bert

     |████████████████████████████████| 57 kB 2.9 MB/s 
     |████████████████████████████████| 96 kB 3.6 MB/s 
     |████████████████████████████████| 6.8 MB 5.5 MB/s 
fatal: destination path 'bert' already exists and is not an empty directory.


## 필요 모듈 불러오기

In [9]:
import os
import sys
import json
import random
import logging

import nltk
import tensorflow as tf
from tensorflow.keras.utils import Progbar


from glob import glob
from google.colab import auth, drive # GCS를 코랩과 연동

# BERT 경로 추가
sys.path.append("bert")

from bert import modeling, optimization, tokenization
from bert.run_pretraining import input_fn_builder, model_fn_builder

In [10]:
auth.authenticate_user()

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



## TPU 설정

In [11]:
# log 정의
logger = logging.getLogger('tensorflow')
logger.setLevel(logging.INFO)
# log format 정의
formatter = logging.Formatter('%(asctime)s : %(message)s')
sh = logging.StreamHandler()
sh.setFormatter(formatter)
logger.handlers = [sh]

# TPU 설정
if 'COLAB_TPU_ADDR' in os.environ:
    logger.info('Using TPU runtime')
    USE_TPU = True 
    TPU_ADDRESS = 'grpc://' + os.environ['COLAB_TPU_ADDR']
    # 세션이 변경되면 tpu 주소가 바뀌기 때문에 설정 
    
    with tf.Session(TPU_ADDRESS) as session: 
        logger.info('TPU address is ' + TPU_ADDRESS) 
        with open('/content/adc.json', 'r') as f: 
            # GCS /content/abc.json에 설정한 과정이 저장되고 이 설정파일로 TPU 사용 
            auth_info = json.load(f) 
        tf.contrib.cloud.configure_gcs(session, credentials=auth_info) 
        
else:
    raise Exception("런타임 -> 런타임 유형변경 -> TPU")

2022-02-23 13:49:57,329 : Using TPU runtime
2022-02-23 13:49:57,332 : TPU address is grpc://10.106.153.218:8470


## 데이터 불러오기

In [12]:
from Korpora import Korpora

Korpora.fetch('kcbert', root_dir = '.')

[kcbert] download kcbert-train.tar.gzaa: 100%|██████████| 2.10G/2.10G [00:41<00:00, 51.0MB/s]
[kcbert] download kcbert-train.tar.gzab: 100%|██████████| 2.10G/2.10G [01:36<00:00, 21.8MB/s]
[kcbert] download kcbert-train.tar.gzac: 671MB [00:08, 75.0MB/s]                           


Unzip tar. It needs a few minutes ... done


In [13]:
# 확인
!ls kcbert/

20190101_20200611_v2.txt  kcbert-train.tar.gzab
kcbert-train.tar.gzaa	  kcbert-train.tar.gzac


In [14]:
# 전처리가 완료된 데이터

check_data = []

with open('kcbert/20190101_20200611_v2.txt', 'r') as f:
    for i in range(10):
        check_data.append(f.readline())

check_data

['우리에게 북한은 꼭 없애야 할 적일뿐\n',
 '문재앙 탄핵 원하면 추천 눌러주세요!! 여론의 힘을 보여줍시다\n',
 '김정은이 트럼프를 개호구 문재인으로 착각했나봄ㅋㅋ\n',
 '부칸이 밑장빼기하다가 딱 걸렸군\n',
 '북한 욕하면 친일이란 그딴 사고방식은 김일성 종합대학에서 배웠니? ㅋㅋ\n',
 '처음부터 북한은 비핵화할 생각이 없다고 보았는데\n',
 'ㅋㅋㅋ 북한 핵돼지 60시간 넘게 개고생하면서 기차 타고 왔는데 트럼프가 빅엿 날리네ㅋㅋㅋ\n',
 '문모씨의 북한팔이도 이제 끝이네요\n',
 '뻔하지머. 트럼프: 비핵화 플랜 내놔라. 김정은: 우선 경제 제재부터 풀어달라. 끝~\n',
 '문재인 작살났네. 개성공단 개박살났네. 금강산 불났네. 남북철도연결은 폭탄맞았네.\n']

## vocab 생성
- 100만개만 학습시켜보자

In [15]:
DEMO_MODE = True

if DEMO_MODE:
    CORPUS_SIZE = 1000000
    !(head -n $CORPUS_SIZE kcbert/20190101_20200611_v2.txt) > dataset.txt
else:
    !mv kcbert/20190101_20200611_v2.txt dataset.txt

In [16]:
!ls -lh dataset.txt

-rw------- 1 root root 144M Feb 23 13:58 dataset.txt


### 데이터셋 나누기
- 4개의 파일에 25만개씩 나누기

- `wordpiece tokenizer`를 통해 vocab을 생성, vocab사이즈는 약 1만개, 단일 글자 최대는 3000개, bert문자는 4개로 생성

- `BertWordPieceTokenizer` 파라미터에서 한국어를 사용할 때, **lowercase = False**를 지정해주는데, 이와 동시에 **strip_accents = False**로 지정해야함. `ㄱ`, `ㅡ`, `ㄴ` 와 같이 단어가 분리된 형태까지 학습되어서 토크나이저 학습이 잘 이루어지지 않는다.

In [17]:
!mkdir ./shards
!split -a 4 -l 256000 -d dataset.txt ./shards/shard_
!ls ./shards/

shard_0000  shard_0001	shard_0002  shard_0003


In [18]:
# huggingface의 Tokenizer 라이브러리로 Tokenizer 생성
from tokenizers import BertWordPieceTokenizer

tokenizer = BertWordPieceTokenizer(lowercase = False, strip_accents = False)
tokenizer.train(['dataset.txt'], vocab_size = 13004, limit_alphabet = 3000)

In [19]:
tokenizer.save_model('.')

['./vocab.txt']

- vocab을 생성할 때, wordpiece, sentencepiece, 형태소분석기 등을 사용할 수 있고, vocab을 생성한 뒤,  Tokenizer로 텍스트를 자른다. 

- 학습된 vocab을 BERT가 이해하려면 `_`로 시작한 토큰을 `##`로 바꿔주고 `[PAD]`, `[UNK]`, `[CLS]`, `[SEP]`, `[MASK]`와 같은 BERT에서 사용하는 특수 토큰 정보를 추가해주어야한다. 이를 `BertWordPieceTokenizer`가 해준뒤 생성된 토큰을 한줄 한줄 저장한 vocab.txt에 저장한다.

In [20]:
!tail -n 20 vocab.txt

우익
주머니
프랑
##공립
##키스탄
반대한다
정도껏
행동은
다니고
네이버는
알아야지
저러니
오르면
소리는
자진
쪽발
철면피
하던지
##미세먼
##가격


## BERT 파라미터 설정

- MAX_SEQ_LENGTH: BERT의 모델 입력의 최장 토큰 길이 이 이상으로는 BERT모델이 이해하지 못한다.
- MASKED_LM_PROB: BERT의 학습 중 Masked LM의 비율을 조정한다.
- MAX_PREDICTIONS: Sequence별 예측할 최대 숫자
- DO_LOWER_CASE: 영문자를 lower(소문자화) 할 지.
- PROCESSES: 전처리할때 CPU 몇개 쓸지
- PRETRAINING_DIR: 프리트레인 데이터 폴더 이름
- VOC_FNAME: 방금 만들어준 vocab.txt 위치

In [21]:
MAX_SEQ_LENGTH = 128
MASKED_LM_PROB = 0.15
MAX_PREDICTIONS = 20
DO_LOWER_CASE = False
PROCESSES = 4
PRETRAINING_DIR = "pretraining_data"
VOC_FNAME = 'vocab.txt'

In [ ]:
XARGS_CMD = ("ls ./shards/ | " "xargs -n 1 -P {} -I{} " "python3 bert/create_pretraining_data.py " "--input_file=./shards/{} " "--output_file={}/{}.tfrecord " "--vocab_file={} " "--do_lower_case={} " "--max_predictions_per_seq={} " "--max_seq_length={} " "--masked_lm_prob={} " "--random_seed=34 " "--dupe_factor=5") 

XARGS_CMD = XARGS_CMD.format(PROCESSES, '{}', '{}', PRETRAINING_DIR, '{}', VOC_FNAME, DO_LOWER_CASE, MAX_PREDICTIONS, MAX_SEQ_LENGTH, MASKED_LM_PROB)

tf.gfile.MkDir(PRETRAINING_DIR)
!$XARGS_CMD



W0223 14:06:40.538288 140427703019392 module_wrapper.py:139] From bert/create_pretraining_data.py:437: The name tf.logging.set_verbosity is deprecated. Please use tf.compat.v1.logging.set_verbosity instead.


W0223 14:06:40.538604 140427703019392 module_wrapper.py:139] From bert/create_pretraining_data.py:437: The name tf.logging.INFO is deprecated. Please use tf.compat.v1.logging.INFO instead.


W0223 14:06:40.538906 140427703019392 module_wrapper.py:139] From /content/gdrive/MyDrive/Colab Notebooks/bert-pretrain-v2/bert/tokenization.py:125: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.




W0223 14:06:40.572365 139939739158400 module_wrapper.py:139] From bert/create_pretraining_data.py:437: The name tf.logging.set_verbosity is deprecated. Please use tf.compat.v1.logging.set_verbosity instead.


W0223 14:06:40.572698 139939739158400 module_wrapper.py:139] From bert/create_pretraining_data.py:437: The name tf.logging.INFO is deprecated. Please use tf.co

In [ ]:
BUCKET_NAME = "자신의 버킷명 입력"
MODEL_DIR = "bert_model"
tf.gfile.MkDir(MODEL_DIR)

assert BUCKET_NAME != ''

출처: https://ebbnflow.tistory.com/162 [Dev Log : 삶은 확률의 구름]